In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=1000, edgeitems=30)
pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 2000)

In [ ]:
# Some StackOverflow magic to enable importing of local module
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from multiLevelCoSurrogates.local import base_dir

In [ ]:
records = os.listdir(base_dir)

print(os.listdir(base_dir))

df = pd.read_csv(base_dir+records[0], index_col='index')

df.reset_index(inplace=True,drop=True)
df.head()


In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12,4))
mse_names = ['mse_high_hier', 'mse_high', 'mse_medium_hier', 'mse_medium', 'mse_low']

sub_df = df.groupby(by='iteration')[mse_names]
means = sub_df.mean()
means[mse_names[:2]].plot(ax=axes[0])
means[mse_names[2:4]].plot(ax=axes[1])
means[mse_names[4]].plot(ax=axes[2])
axes[2].legend(loc=1)

low_bounds, high_bounds = sub_df.quantile(.05).values.T, sub_df.quantile(.95).values.T
for low, high, color in zip(low_bounds, high_bounds, ['C0', 'C1', 'C2', 'C3']):
    ax.fill_between(means.index, low, high, color=color, alpha=.2)

# ax.set_title('acq on {}, {} fidelity updated'.format(*name))

plt.tight_layout()
plt.show()

These plots show the raw MSE values (and 95% CI) for the three cases using the 'Expected Improvement' acquisition function. In the last plot, only the high fidelity model is updated, so the other values are constant.

In [ ]:
def plot_normalized(df, axes=None, save_as=None):
    mse_names = ['mse_high_hier', 'mse_high', 'mse_medium_hier', 'mse_medium', 'mse_low']

    mse_groups =[
        ['mse_high_hier', 'mse_high'], 
        ['mse_medium_hier', 'mse_medium'],
        ['mse_low'],
    ]
    
    df.reset_index(inplace=True,drop=True)
    
    if axes is None:
        fig, axes = plt.subplots(ncols=3, figsize=(12,4))
    
    # Separate out the MSE values after initial training to use as reference
    num_iters = len(df['iteration'].unique())
    
    for mse_group, ax in zip(mse_groups, axes):
        first_rows = df.loc[0::num_iters,mse_group].values

        # Normalize the values in the dataframe and plot them
        df.loc[:,mse_group] = df.loc[:,mse_group].values / np.mean(first_rows)
        sub_df = df.groupby(by='iteration')[mse_group]
        means = sub_df.mean()
        means[mse_group].plot(ax=ax)
        ax.legend(loc=1)

        low_bounds, high_bounds = sub_df.quantile(.05).values.T, sub_df.quantile(.95).values.T
        for low, high, color in zip(low_bounds, high_bounds, ['C0', 'C1', 'C2', 'C3', 'C4']):
            ax.fill_between(means.index, low, high, color=color, alpha=.2)

        ax.axhline(y=1.0, color='black', alpha=0.5)
    #     ax.set_ylim([0,2])
    #     ax.set_yscale('log')

    plt.tight_layout()
    if save_as:
        plt.savefig(save_as)
    plt.show()
    
plot_normalized(pd.read_csv(base_dir+records[0], index_col='index')) #, save_as='plots/EI-mse.pdf')

The following plots are all normalized to a starting MSE of '1.0', which is comparable as they are all initialized with just 3 high fidelity points and 5 low fidelity points.

In [ ]:
for rec in records:
    print(rec)
    plot_normalized(pd.read_csv(base_dir+rec, index_col='index'), save_as=f"plots/{rec.split('.')[0]}.png")

# print(records[0])
# plot_normalized(pd.read_csv(base_dir+records[0], index_col='index')) #, save_as='plots/EI-mse.pdf')

In [ ]:
print(records[1])
plot_normalized(pd.read_csv(base_dir+records[1], index_col='index')) #, save_as='plots/UCB-mse.pdf')

In [ ]:
df = pd.read_csv(base_dir+records[0], index_col='index')
# mse_names = ['mse_high', 'mse_low', 'mse_hier', 'mse_low_on_high']
mse_names = ['mse_high', 'mse_hier']

# Separate out the MSE values after initial training to use as reference
num_iters = len(df['iteration'].unique())
first_rows = df.loc[0::num_iters,mse_names].values

# Normalize the values in the dataframe
df.loc[:,mse_names] = df.loc[:,mse_names].values / np.mean(first_rows)



b3hier = df[df['which_model'] == 'hierarchical']
b3hier = b3hier[b3hier['fidelity'] == 'both 3']
b3hier = b3hier.groupby(by='iteration')[['mse_high', 'mse_hier']]

highhigh = df[df['which_model'] == 'high']
highhigh = highhigh[highhigh['fidelity'] == 'high']
highhigh = highhigh.groupby(by='iteration')['mse_high']

fig = plt.figure(figsize=(12,9))
plt.plot(b3hier.mean()['mse_high'], label='both 3, hierarchical: mse_high')
plt.plot(b3hier.mean()['mse_hier'], label='both 3, hierarchical: mse_hier')
plt.plot(np.repeat(highhigh.mean().values, 3), label='high, high: mse_high')

low_bounds, high_bounds = b3hier.quantile(.05).values.T, b3hier.quantile(.95).values.T
plt.fill_between(b3hier.mean().index, low_bounds[0], high_bounds[0], color='C0', alpha=.2)
plt.fill_between(b3hier.mean().index, low_bounds[1], high_bounds[1], color='C1', alpha=.2)

low_bounds, high_bounds = highhigh.quantile(.05).values.T, highhigh.quantile(.95).values.T
plt.fill_between(np.arange(len(low_bounds))*3, low_bounds, high_bounds, color='C2', alpha=.2)

plt.axhline(y=1.0, alpha=0.33, color='black')
plt.legend(loc=0)
# plt.yscale('log')
# plt.savefig('plots/equal-scaled-MSE.pdf')
plt.show()



Here we focus on the best performing cases: acquisition function (expected improvement) on high fidelity only and hierarchical prediction.

In the above plot, the 'update high fidelity based on high fidelity acquisition function' plot has been stretched by a factor of 3 to match up with the number of high fidelity evaluations in the 'both every 3rd on hierarchical acquisition function' case.
It can be seen that, according to MSE, the hierarchical model (mse_hier) is actually better than the high fidelity model, with both of them ourperforming the single-fidelity (high, high) case.

The shaded areas show the 5th to 95th percentile of the MSE results. The spread is more varied for the bi-fidelity case at the start, but seems to become more stable towards the end of the optimization.